In [1]:
from google.colab import drive

drive.mount('/content/drive')
!ln -s /content/drive/MyDrive/ /my_drive
!ls
!ls /my_drive

Mounted at /content/drive
drive  sample_data
'Colab Notebooks'
 How_CNNs_work.pdf
'How neural networks work.pdf'
 interview
'Introducing-Bluetooth-LE-Audio-book - Copy.pdf'
 machine_leanring_course
 MyDrive
 ntust_course
'ParC-Net: Position Aware Circular Convolution with Merits from ConvNets and Transformer.md'
 windows_linux_share


In [2]:
# download Apple Scab Detection
!git clone https://github.com/AngusWooster/edge_ai_apple_scab_detection.git
%cd edge_ai_apple_scab_detection/

Cloning into 'edge_ai_apple_scab_detection'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 142 (delta 5), reused 95 (delta 4), pack-reused 46
Receiving objects: 100% (142/142), 51.41 MiB | 48.39 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/edge_ai_apple_scab_detection


In [13]:
# Install requirement package
!pip install openvino
!pip install openvino-dev
!pip install gdown
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 61.0 MB/s eta 0:00:00
  Created wheel for jstyleson: filename=jstyleson-0.0.2-py3-none-any.whl size=2384 sha256=31d3f6f16fdac07bafabd1e92cd2f4a80f85e8a6fb3138e235459d7dfd18cf30
  Stored in directory: /root/.cache/pip/wheels/12/51/c6/a1e751db88203e11c6d9ffe4683ca3d8c14b1479639bec1006
Successfully built jstyleson
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!mkdir -p faster_rcnn/outputs/training/final_dataset
!mkdir -p yolov7/weight/final_dataset

In [6]:
%cd faster_rcnn/outputs/training/final_dataset
# Download faster_rcnn pre-trained weight
!gdown https://drive.google.com/uc?id=1S6LizlqZSI8zf4Blsq8gmwFHtPp81U5_


%cd ../../../../
%cd yolov7/weight/final_dataset/
# Download yolov7 pre-trained weight
!gdown https://drive.google.com/uc?id=1amTSXUGkdwDn96tr62obRfftmVfmxI_u

%cd ../../

/content/edge_ai_apple_scab_detection/faster_rcnn/outputs/training/final_dataset
Downloading...
From: https://drive.google.com/uc?id=1S6LizlqZSI8zf4Blsq8gmwFHtPp81U5_
To: /content/edge_ai_apple_scab_detection/faster_rcnn/outputs/training/final_dataset/best_model.pth
100% 173M/173M [00:03<00:00, 55.0MB/s]
/content/edge_ai_apple_scab_detection
/content/edge_ai_apple_scab_detection/yolov7/weight/final_dataset
Downloading...
From: https://drive.google.com/uc?id=1amTSXUGkdwDn96tr62obRfftmVfmxI_u
To: /content/edge_ai_apple_scab_detection/yolov7/weight/final_dataset/best.pt
100% 74.8M/74.8M [00:00<00:00, 87.0MB/s]
/content/edge_ai_apple_scab_detection/yolov7


# Export model to onnx 

In [11]:
# !python3 export.py --help
!python3 -W ignore export.py --weights weight/final_dataset/best.pt --grid

Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(weights='weight/final_dataset/best.pt', img_size=[640, 640], batch_size=1, dynamic=False, dynamic_batch=False, grid=True, end2end=False, max_wh=None, topk_all=100, iou_thres=0.45, conf_thres=0.25, device='cpu', simplify=False, include_nms=False, fp16=False, int8=False)
YOLOR 🚀 774d1e9 torch 2.0.1+cu118 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients

Starting TorchScript export with torch 2.0.1+cu118...
TorchScript export success, saved as weight/final_dataset/best.torchscript.pt
CoreML export failure: No module named 'coremltools'

Starting TorchScript-Lite export with torch 2.0.1+cu118...
TorchScript-Lite export success, saved as weight/final_dataset/best.torchscript.ptl

Starting ONNX export with onnx 1.14.0...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu1

In [14]:
import sys
import time
from pathlib import Path
from typing import List, Tuple, Dict
import os
import glob
import cv2
import matplotlib.pyplot as plt
from utils.general import scale_coords, non_max_suppression
from openvino.runtime import Model
from openvino.tools import mo
from openvino.runtime import Core

In [15]:
model = mo.convert_model('weight/final_dataset/best.onnx')
core = Core()
# load model on CPU device
compiled_model = core.compile_model(model, 'CPU')

In [17]:
import numpy as np
import torch
from PIL import Image
from utils.datasets import letterbox
from utils.plots import plot_one_box


def preprocess_image(img0: np.ndarray):
    """
    Preprocess image according to YOLOv7 input requirements. 
    Takes image in np.array format, resizes it to specific size using letterbox resize, converts color space from BGR (default in OpenCV) to RGB and changes data layout from HWC to CHW.
    
    Parameters:
      img0 (np.ndarray): image for preprocessing
    Returns:
      img (np.ndarray): image after preprocessing
      img0 (np.ndarray): original image
    """
    # resize
    img = letterbox(img0, auto=False)[0]
    
    # Convert
    img = img.transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    return img, img0


def prepare_input_tensor(image: np.ndarray):
    """
    Converts preprocessed image to tensor format according to YOLOv7 input requirements. 
    Takes image in np.array format with unit8 data in [0, 255] range and converts it to torch.Tensor object with float data in [0, 1] range
    
    Parameters:
      image (np.ndarray): image for conversion to tensor
    Returns:
      input_tensor (torch.Tensor): float tensor ready to use for YOLOv7 inference
    """
    input_tensor = image.astype(np.float32)  # uint8 to fp16/32
    input_tensor /= 255.0  # 0 - 255 to 0.0 - 1.0
    
    if input_tensor.ndim == 3:
        input_tensor = np.expand_dims(input_tensor, 0)
    return input_tensor


# label names for visualization
NAMES = ['scab']

# colors for visualization
# COLORS = {name: [np.random.randint(0, 255) for _ in range(3)]
#           for i, name in enumerate(NAMES)}
COLORS = {'scab': [255, 0, 0]}

In [18]:
def detect(model: Model, image_path: Path, conf_thres: float = 0.25, iou_thres: float = 0.45, classes: List[int] = None, agnostic_nms: bool = False):
    """
    OpenVINO YOLOv7 model inference function. Reads image, preprocess it, runs model inference and postprocess results using NMS.
    Parameters:
        model (Model): OpenVINO compiled model.
        image_path (Path): input image path.
        conf_thres (float, *optional*, 0.25): minimal accpeted confidence for object filtering
        iou_thres (float, *optional*, 0.45): minimal overlap score for remloving objects duplicates in NMS
        classes (List[int], *optional*, None): labels for prediction filtering, if not provided all predicted labels will be used
        agnostic_nms (bool, *optiona*, False): apply class agnostinc NMS approach or not
    Returns:
       pred (List): list of detections with (n,6) shape, where n - number of detected boxes in format [x1, y1, x2, y2, score, label] 
       orig_img (np.ndarray): image before preprocessing, can be used for results visualization
       inpjut_shape (Tuple[int]): shape of model input tensor, can be used for output rescaling
    """
    output_blob = model.output(0)
    img = np.array(Image.open(image_path))
    preprocessed_img, orig_img = preprocess_image(img)
    input_tensor = prepare_input_tensor(preprocessed_img)
    t1 = time.time()
    predictions = torch.from_numpy(model(input_tensor)[output_blob])
    model_inference_time = time.time() - t1
    pred = non_max_suppression(predictions, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms)
    return pred, orig_img, input_tensor.shape, model_inference_time


def draw_boxes(predictions: np.ndarray, input_shape: Tuple[int], image: np.ndarray, names: List[str], colors: Dict[str, int]):
    """
    Utility function for drawing predicted bounding boxes on image
    Parameters:
        predictions (np.ndarray): list of detections with (n,6) shape, where n - number of detected boxes in format [x1, y1, x2, y2, score, label]
        image (np.ndarray): image for boxes visualization
        names (List[str]): list of names for each class in dataset
        colors (Dict[str, int]): mapping between class name and drawing color
    Returns:
        image (np.ndarray): box visualization result
    """
    if not len(predictions):
        return image
    # Rescale boxes from input size to original image size
    predictions[:, :4] = scale_coords(input_shape[2:], predictions[:, :4], image.shape).round()

    # Write results
    for *xyxy, conf, cls in reversed(predictions):
        label = f'{names[int(cls)]} {conf:.2f}'
        plot_one_box(xyxy, image, label=label, color=colors[names[int(cls)]], line_thickness=3)
    return image

In [19]:
def openvino_inference_dir_create(dir_name = "eval") -> str:
    """
    This functions counts the number of evaluation directories already present
    and creates a new one in `outputs/evaluation/`.
    And returns the directory path.
    """
    dir_path = 'outputs/openvino_inference' + '/' + dir_name
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    num_eval_dirs_present = len(os.listdir(dir_path))
    next_dir_num = num_eval_dirs_present + 1
    new_dir_path = f"{dir_path}/res_{next_dir_num}"
    os.makedirs(new_dir_path, exist_ok=True)
    return new_dir_path

In [20]:
%matplotlib inline
validation_image_path = "../dataset/final_dataset/images/valid/"
image_file_types = ['*.jpg', '*.jpeg', '*.JPG']
all_image_paths = []

output_dir = openvino_inference_dir_create()
print(f"inference output: {output_dir}")

for file_type in image_file_types:
    all_image_paths.extend(sorted(glob.glob(os.path.join(validation_image_path, file_type))))

for i, img_path in enumerate(all_image_paths):
    boxes, image, input_shape, model_inference_time = detect(compiled_model, img_path)
    image_with_boxes = draw_boxes(boxes[0], input_shape, image, NAMES, COLORS)
    if i == 0:
        plt.imshow(image_with_boxes)
        plt.show()
    # save image
    image = cv2.cvtColor(image_with_boxes, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"{output_dir}/image_{i}.jpg", image)
    print(f"{i:03d}: model_inference = ({(1E3 * (model_inference_time)):.1f}ms)")
print(f"done")

000: model_inference = (1593.6ms)
001: model_inference = (1549.3ms)
002: model_inference = (1227.3ms)
003: model_inference = (1196.6ms)
004: model_inference = (1192.0ms)
005: model_inference = (1198.5ms)
006: model_inference = (1199.2ms)
007: model_inference = (1192.5ms)
008: model_inference = (1209.8ms)
009: model_inference = (1197.2ms)
010: model_inference = (1395.0ms)
011: model_inference = (1494.9ms)
012: model_inference = (1522.8ms)
013: model_inference = (1639.6ms)
014: model_inference = (1209.8ms)
015: model_inference = (1207.0ms)
016: model_inference = (1198.7ms)
017: model_inference = (1205.9ms)
018: model_inference = (2047.9ms)
019: model_inference = (2589.9ms)
020: model_inference = (3979.3ms)
021: model_inference = (2023.1ms)
022: model_inference = (1212.2ms)
023: model_inference = (1214.5ms)
024: model_inference = (1195.9ms)
025: model_inference = (1191.6ms)
026: model_inference = (1384.2ms)
027: model_inference = (1492.5ms)
028: model_inference = (1526.0ms)
029: model_inf